# ConversationKGMemory

## 概覽

與以鍵值格式管理個別實體信息的 ```ConversationEntityMemory``` 不同，```ConversationKGMemory```（對話知識圖記憶體）是一個以圖形格式管理實體之間關係的模組。

它提取並結構化**知識三元組**（主詞-關係-受詞）來識別和存儲實體之間的複雜關係，並允許通過**圖結構**探索實體連接性。

這有助於模型理解不同實體之間的關係，並基於複雜網絡和歷史上下文更好地回應查詢。

## 目錄

- [概覽](#概覽)
- [環境設定](#環境設定)
- [對話知識圖記憶體](#對話知識圖記憶體)
- [將 KG 記憶體應用於鏈](#將-kg-記憶體應用於鏈)
- [使用 LCEL 應用 KG 記憶體](#使用-lcel-應用-kg-記憶體)

## 參考資料

- [LangChain Python API Reference>langchain-community: 0.3.13>memory>ConversationKGMemory](https://python.langchain.com/api_reference/community/memory/langchain_community.memory.kg.ConversationKGMemory.html)
- [LangChain Python API Reference>langchain-community: 0.2.16>NetworkxEntityGraph](https://python.langchain.com/v0.2/api_reference/community/graphs/langchain_community.graphs.networkx_graph.NetworkxEntityGraph.html#langchain_community.graphs.networkx_graph.NetworkxEntityGraph)

---

**詳細解釋：**

### ConversationKGMemory 的核心概念

**1. 知識圖譜架構**
- 不同於簡單的實體-資訊對應，KG Memory 建立了完整的實體關係網絡
- 例如：「張三」-「工作於」-「科技公司」、「張三」-「居住在」-「台北」
- 這些關係形成一個可查詢和遍歷的圖結構

**2. 知識三元組（Knowledge Triplets）**
- **主詞（Subject）**：關係的發起者（如：張三）
- **關係（Relation）**：連接兩個實體的動作或狀態（如：工作於、住在）
- **受詞（Object）**：關係的接受者（如：公司、城市）

**3. 相較於 ConversationEntityMemory 的優勢**
- **ConversationEntityMemory**：僅儲存個別實體的屬性
  - 例如：張三 → {職業：工程師, 年齡：30}
- **ConversationKGMemory**：儲存實體間的複雜關係
  - 例如：張三 ← 同事 → 李四 ← 朋友 → 王五

**4. 實際應用場景**
- **客戶關係管理**：追蹤客戶之間的推薦關係
- **企業組織**：理解員工之間的報告關係和協作網絡
- **學術研究**：建立研究者、機構、論文之間的引用關係
- **社交網絡分析**：分析用戶之間的互動和影響關係

**5. 圖結構的查詢能力**
- 可以回答如「誰是張三的同事？」「哪些人與某個專案有關？」
- 支援多跳查詢：「張三的同事的朋友有誰？」
- 能夠發現隱含的關係模式和網絡效應

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "05-ConversationKGMemory",  # title 과 동일하게 설정해 주세요
    }
)

Environment variables have been set successfully.


You can alternatively set API keys such as ```OPENAI_API_KEY``` in a ```.env``` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [4]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

## 對話知識圖記憶體

```ConversationKGMemory``` 是一個記憶體模組，以圖結構存儲和管理從對話中提取的信息。

此範例演示以下主要功能：
- 存儲對話上下文（```save_context```）
- （參考）取得圖中按因果依賴排序的實體名稱列表（```get_topological_sort```）
- 從當前對話中提取實體（```get_current_entities```）
- 提取知識三元組（```get_knowledge_triplets```）
- 檢索存儲的記憶體（```load_memory_variables```）

以下範例展示了從關於新設計師 Shelly Kim 的對話中提取實體和關係，並以圖形格式存儲的過程。

**詳細說明：**

這個範例將示範 ConversationKGMemory 如何處理一個具體的對話場景。當我們討論新設計師 Shelly Kim 時，系統會：

1. **識別實體**：從對話中自動識別關鍵實體（如：Shelly Kim、設計師、公司等）

2. **建立關係**：分析實體之間的關係
   - 例如：Shelly Kim → 職業 → 設計師
   - Shelly Kim → 工作地點 → 某公司

3. **圖形存儲**：將這些關係以圖的形式存儲，便於後續查詢和推理

4. **拓撲排序**：按照實體之間的依賴關係進行排序，幫助理解信息的邏輯順序

5. **三元組提取**：將複雜的對話內容轉換為結構化的「主詞-關係-受詞」格式

這種方法特別適用於需要記住人物關係、組織結構、或任何涉及多個實體間複雜互動的對話場景。

In [5]:
from langchain_openai import ChatOpenAI
from langchain_community.memory.kg import ConversationKGMemory

In [6]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "This is Shelly Kim who lives in Pangyo."},
    {"output": "Hello Shelly, nice to meet you! What kind of work do you do?"},
)
memory.save_context(
    {"input": "Shelly Kim is our company's new designer."},
    {
        "output": "That's great! Welcome to our team. I hope you'll enjoy working with us."
    },
)

### （參考）get_topological_sort() → List[str]

您可以使用 ```get_topological_sort``` 方法以拓撲順序查看知識圖中存儲的所有實體：

此方法：
- 使用 ```NetworkX``` 函式庫分析知識圖結構。
- 基於有向邊執行拓撲排序。
- 返回按依賴順序排列的實體列表。

該順序反映了對話中實體之間的關係，顯示它們在知識圖中的連接方式。

**深入解析：**

**拓撲排序的核心原理：**
- **有向無環圖（DAG）**：知識圖必須是無環的才能進行拓撲排序
- **依賴關係**：如果實體 A 指向實體 B，則 A 在排序中會出現在 B 之前
- **層次結構**：幫助識別實體在知識網絡中的層級和重要性

**NetworkX 的角色：**
- **圖分析專家**：提供強大的圖論算法實現
- **效率優化**：針對大型圖結構進行了性能優化
- **豐富功能**：除了拓撲排序外，還提供路徑查找、中心性分析等功能

**實際應用場景：**

1. **企業組織架構**
   ```
   CEO → 副總裁 → 部門經理 → 組長 → 員工
   ```

2. **項目依賴關係**
   ```
   需求分析 → 設計 → 開發 → 測試 → 部署
   ```

3. **學術引用網絡**
   ```
   基礎理論 → 應用研究 → 實證研究 → 案例分析
   ```

**排序結果的意義：**
- **前序實體**：通常是根源、原因或基礎概念
- **後序實體**：通常是結果、效應或衍生概念
- **中間實體**：起到橋樑作用，連接不同層級的概念

**技術優勢：**
- **循環檢測**：自動識別並處理可能的循環依賴
- **多路徑分析**：處理複雜的多重依賴關係
- **可視化支援**：便於將抽象的關係圖形化展示

In [7]:
memory.kg.get_topological_sort()

['Shelly Kim', 'Pangyo', "our company's new designer"]

### get_current_entities(input_string: str) → List[str]

以下是 ```get_current_entities``` 方法的運作方式：

**1. 實體提取鏈創建**
- 使用 ```entity_extraction_prompt``` 模板創建 ```LLMChain```。
- 此提示專門設計用於從對話的最後一行提取專有名詞。

**2. 上下文處理**
- 從緩衝區檢索最後 **k*2** 條訊息。（預設：k=2）
- 使用 ```human_prefix``` 和 ```ai_prefix``` 生成對話歷史字串。

**3. 實體提取**
- 從輸入字串「Who is Shelly Kim?」中提取專有名詞
- 主要識別以大寫字母開頭的單詞作為專有名詞。
- 在此情況下，「Shelly Kim」被提取為實體。

此方法**僅從問題本身提取實體**，而先前的對話上下文僅用作參考。

**詳細說明：**

**實體提取的工作原理：**
1. **語言模型驅動**：使用 LLM 的自然語言理解能力來識別實體
2. **專有名詞識別**：重點關注人名、地名、公司名等專有名詞
3. **上下文感知**：雖然主要分析當前輸入，但會參考歷史對話來提高準確性

**k*2 訊息的意義：**
- k 代表對話輪次數（預設為 2）
- 乘以 2 是因為每輪對話包含人類訊息和 AI 回應
- 總共檢索 4 條訊息作為上下文參考

**實體識別規則：**
- **大寫字母開頭**：通常表示專有名詞
- **語義分析**：LLM 會根據語境判斷是否為實體
- **領域相關**：可能識別特定領域的術語和概念

**方法的限制：**
- 主要依賴當前輸入，可能遺漏歷史對話中的重要實體
- 對於複雜的實體關係，可能需要更深入的上下文分析
- 語言和格式的變化可能影響識別準確性

In [8]:
memory.get_current_entities({"input": "Who is Shelly Kim?"})

['Shelly Kim']

### get_knowledge_triplets(input_string: str) → List[KnowledgeTriple]

```get_knowledge_triplets``` 方法的運作方式如下：

**1. 知識三元組提取鏈**
- 使用 ```knowledge_triplet_extraction_prompt``` 模板創建 ```LLMChain```。
- 專門設計用於從給定文本中提取（**主詞-關係-受詞**）格式的三元組。

**2. 記憶體搜尋**
- 從先前存儲的對話中搜尋與「Shelly」相關的信息。
- 存儲的上下文：
  - 「This is Shelly Kim who lives in Pangyo.」
  - 「Shelly Kim is our company's new designer.」

**3. 三元組提取**
- 從檢索到的信息中生成以下三元組：
  - (Shelly Kim, lives in, Pangyo)
  - (Shelly Kim, is, designer)
  - (Shelly Kim, works at, our company)

此方法從所有與**特定實體相關**的存儲對話內容中提取**三元組格式**的關係信息。

**深入說明：**

**知識三元組的結構：**
- **主詞（Subject）**：關係的發起者，通常是人、物或概念
- **關係（Relation）**：描述主詞和受詞之間連接的動詞或狀態
- **受詞（Object）**：關係的接受者或目標

**提取過程的技術細節：**

1. **語義分析**
   - LLM 理解自然語言中的隱含關係
   - 識別同義詞和相似表達（如「works at」和「is employed by」）

2. **實體消歧**
   - 確保「Shelly」和「Shelly Kim」指向同一實體
   - 處理代詞和簡稱的歸一化

3. **關係標準化**
   - 將多樣的自然語言表達轉換為標準化的關係標籤
   - 例如：「住在」、「居住於」→ 統一為「lives in」

**實際應用價值：**
- **結構化知識**：將非結構化對話轉換為可查詢的知識庫
- **關係推理**：支援複雜的多跳查詢和推理
- **知識圖譜建構**：為大型知識圖譜提供基礎數據

**輸出格式 KnowledgeTriple：**
```python
class KnowledgeTriple:
    subject: str      # 主詞
    predicate: str    # 關係/謂詞
    object: str       # 受詞
```

這種結構化的表示方式使得知識能夠被有效存儲、查詢和推理使用。

In [9]:
memory.get_knowledge_triplets({"input": "Shelly"}), "\n", memory.get_knowledge_triplets(
    {"input": "Pangyo"}
), "\n", memory.get_knowledge_triplets(
    {"input": "designer"}
), "\n", memory.get_knowledge_triplets(
    {"input": "Langchain"}
)

([KnowledgeTriple(subject='Shelly Kim', predicate='lives in', object_='Pangyo'),
  KnowledgeTriple(subject='Shelly Kim', predicate='is', object_="company's new designer")],
 '\n',
 [KnowledgeTriple(subject='Shelly Kim', predicate='lives in', object_='Pangyo')],
 '\n',
 [KnowledgeTriple(subject='Shelly Kim', predicate='is a', object_='designer')],
 '\n',
 [])

### load_memory_variables(inputs: Dict[str, Any]) → Dict[str, Any]

```load_memory_variables``` 方法通過以下步驟運作：

**1. 實體提取**
- 從輸入「Who is Shelly Kim?」中提取實體（例如「Shelly Kim」）
- 內部使用 ```get_current_entities``` 方法。

**2. 知識檢索**
- 搜尋與提取實體相關的所有知識三元組。
- 查詢圖形中先前通過 ```save_context``` 方法存儲的信息。

**3. 信息格式化**
- 將找到的三元組轉換為系統訊息。
- 由於 ```return_messages=True``` 設定，返回訊息物件列表。

此方法從存儲的知識圖中檢索相關信息，並以結構化格式返回，隨後可用作與語言模型進行後續對話的上下文。

**詳細流程說明：**

**步驟 1：實體識別與提取**
```python
# 輸入處理
input_text = "Who is Shelly Kim?"
entities = get_current_entities(input_text)  # 返回 ["Shelly Kim"]
```

**步驟 2：圖形查詢過程**
```python
# 知識圖搜尋
for entity in entities:
    related_triplets = search_graph(entity)
    # 例如找到：
    # (Shelly Kim, lives in, Pangyo)
    # (Shelly Kim, is, designer)
    # (Shelly Kim, works at, our company)
```

**步驟 3：結果格式化**
```python
# 轉換為訊息格式
messages = []
for triplet in triplets:
    message = SystemMessage(
        content=f"{triplet.subject} {triplet.predicate} {triplet.object}"
    )
    messages.append(message)
```

**return_messages=True 的影響：**
- **True**：返回 `List[BaseMessage]` 物件，適合直接傳入聊天模型
- **False**：返回純文字字串，適合傳統的文字處理

**在對話鏈中的應用：**
1. **上下文注入**：將檢索到的知識作為系統訊息注入對話
2. **個性化回應**：基於已知實體信息提供更準確的回答
3. **知識連貫性**：確保對話中的信息與先前存儲的知識保持一致

**技術優勢：**
- **動態檢索**：只檢索與當前查詢相關的信息
- **結構化輸出**：便於後續處理和整合
- **擴展性**：支援複雜的多實體查詢和關係推理

In [10]:
memory.load_memory_variables({"input": "Who is Shelly Kim?"})

{'history': [SystemMessage(content="On Shelly Kim: Shelly Kim lives in Pangyo. Shelly Kim is our company's new designer.", additional_kwargs={}, response_metadata={})]}

## Applying KG Memory to Chain

This section demonstrates how to use ```ConversationKGMemory``` with **ConversationChain** .

(The class **ConversationChain** was deprecated in LangChain 0.2.7 and will be removed in 1.0. If you want, you can skip to [Applying KG Memory with LCEL](#applying-kg-memory-with-lcel))

In [11]:
from langchain_community.memory.kg import ConversationKGMemory
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

C:\Users\Caelu\AppData\Local\Temp\ipykernel_5648\1729312250.py:21: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_with_kg = ConversationChain(


Let's initialize the conversation with some basic information.

In [12]:
conversation_with_kg.predict(
    input="My name is Teddy. Shelly is a coworker of mine, and she's a new designer at our company."
)

"Hi Teddy! It's great to meet you. It sounds like you and Shelly are working together in a creative environment. Being a new designer, Shelly must be bringing fresh ideas and perspectives to your team. How has it been working with her so far?"

Let's query the memory for information about Shelly.

In [13]:
conversation_with_kg.memory.load_memory_variables({"input": "who is Shelly?"})

{'history': 'On Shelly: Shelly is a coworker of Teddy. Shelly is a new designer. Shelly works at our company.'}

You can also reset the memory by ```memory.clear()```.

In [14]:
conversation_with_kg.memory.clear()
conversation_with_kg.memory.load_memory_variables({"input": "who is Shelly?"})

{'history': ''}

## Applying KG Memory with LCEL

Let's examine the memory after having a conversation using a custom **ConversationChain** with ```ConversationKGMemory``` by LCEL

In [15]:
from operator import itemgetter
from langchain_community.memory.kg import ConversationKGMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:
{history}""",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationKGMemory(llm=llm, return_messages=True, memory_key="history")


class ConversationChain:
    def __init__(self, prompt, llm, memory):
        self.memory = memory
        self.chain = (
            RunnablePassthrough()
            | RunnablePassthrough.assign(
                history=RunnableLambda(memory.load_memory_variables)
                | itemgetter("history")
            )
            | prompt
            | llm
        )

    def invoke(self, input_dict):
        response = self.chain.invoke(input_dict)
        self.memory.save_context(input_dict, {"output": response.content})
        return response


conversation_with_kg = ConversationChain(prompt, llm, memory)

Let's initialize the conversation with some basic information.

In [16]:
response = conversation_with_kg.invoke(
    {
        "input": "My name is Teddy. Shelly is a coworker of mine, and she's a new designer at our company."
    }
)
response.content

"Hi Teddy! It's nice to meet you. It sounds like you and Shelly are working together at your company. How's everything going with the new designer on board?"

Let's query the memory for information about Shelly.

In [17]:
conversation_with_kg.memory.load_memory_variables({"input": "who is Shelly?"})

{'history': [SystemMessage(content='On Shelly: Shelly is a coworker of Teddy. Shelly is a new designer. Shelly works at our company.', additional_kwargs={}, response_metadata={})]}

You can also reset the memory by ```memory.clear()```.

In [18]:
conversation_with_kg.memory.clear()
conversation_with_kg.memory.load_memory_variables({"input": "who is Shelly?"})

{'history': []}